In [2]:
import os

os.sys.path.insert(0, '../script')

import webnlg

In [3]:
webnlg.DATASETS_FILEPATHS.keys()

dict_keys(['dev', 'train', 'test_no_lex', 'test_unseen_with_lex', 'test_with_lex', 'dev_1.2', 'train_1.2'])

In [4]:
corpus = webnlg.load(dataset=['dev', 'train', 'test_with_lex'])

In [21]:
sample = corpus.sample(seed=300)

sample

Triple info: category=Food eid=Id213

	Modified triples:

Philippines | ethnicGroup | Ilocano_people
Philippines | language | Arabic
Philippines | ethnicGroup | Zamboangans
Philippines | language | Philippine_Spanish
Batchoy | country | Philippines


	Lexicalizations:

Batchoy is eaten in the Philippines where Philippine Spanish and Arabic are spoken. It is also where the Zamboangans and Ilocano people are two of the ethnic groups.


Zamboangans and Ilocano people are ethnic groups from the Philippines which is home to the dish batchoy and where Philippine Spanish and Arabic are spoken.


Batchoy comes from the Philippines, where Philippine Spanish and Arabic are among the languages spoken, and the Zamboangans and IIocano people are ethnic groups.


In [5]:
def pairing(iterable):
    
    for i, item1 in enumerate(iterable):
        
        for j, item2 in enumerate(iterable):
            
            if i != j:
                
                yield (item1, item2)

In [26]:
list(pairing(sample.lexes()))

[('Batchoy is eaten in the Philippines where Philippine Spanish and Arabic are spoken. It is also where the Zamboangans and Ilocano people are two of the ethnic groups.',
  'Zamboangans and Ilocano people are ethnic groups from the Philippines which is home to the dish batchoy and where Philippine Spanish and Arabic are spoken.'),
 ('Batchoy is eaten in the Philippines where Philippine Spanish and Arabic are spoken. It is also where the Zamboangans and Ilocano people are two of the ethnic groups.',
  'Batchoy comes from the Philippines, where Philippine Spanish and Arabic are among the languages spoken, and the Zamboangans and IIocano people are ethnic groups.'),
 ('Zamboangans and Ilocano people are ethnic groups from the Philippines which is home to the dish batchoy and where Philippine Spanish and Arabic are spoken.',
  'Batchoy is eaten in the Philippines where Philippine Spanish and Arabic are spoken. It is also where the Zamboangans and Ilocano people are two of the ethnic groups

In [44]:
with open('paraphrases.txt', 'w') as f:
    
    f.write('input\toutput\n')
    
    i = 0
    
    for entry in corpus:
        
        for pair in pairing(entry.lexes()):
            
            f.write('{}\t{}\n'.format(*pair))
            i += 1

# Delexicalized

In [45]:
delex = webnlg.load(dataset=['dev_1.2', 'train_1.2'])

with open('paraphrases_delex.txt', 'w') as f:
    
    f.write('input\toutput\n')
    
    i = 0
    
    for entry in delex:
        
        for pair in pairing([l['template'] for l in entry.entry['lexes']]):
            
            f.write('{}\t{}\n'.format(*pair))
            i += 1

In [46]:
!head paraphrases_delex.txt

input	output
AGENT-1 was created by PATIENT-1 .	The creator of AGENT-1 is PATIENT-1 .
AGENT-1 was created by PATIENT-1 .	AGENT-1 , was created by PATIENT-1 .
The creator of AGENT-1 is PATIENT-1 .	AGENT-1 was created by PATIENT-1 .
The creator of AGENT-1 is PATIENT-1 .	AGENT-1 , was created by PATIENT-1 .
AGENT-1 , was created by PATIENT-1 .	AGENT-1 was created by PATIENT-1 .
AGENT-1 , was created by PATIENT-1 .	The creator of AGENT-1 is PATIENT-1 .
AGENT-1 was created by PATIENT-1 .	AGENT-1 was created by PATIENT-1 .
AGENT-1 was created by PATIENT-1 .	AGENT-1 was created by PATIENT-1 .
AGENT-1 was created by PATIENT-1 .	AGENT-1 was created by PATIENT-1 .


# OpenNMT

In [1]:
!pip install OpenNMT-tf[tensorflow_gpu]

    100% |████████████████████████████████| 153kB 1.0MB/s ta 0:00:01
    100% |████████████████████████████████| 276kB 1.1MB/s ta 0:00:01
  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))': /simple/rouge/
    100% |████████████████████████████████| 1.8MB 4.3MB/s eta 0:00:01
    100% |████████████████████████████████| 345.2MB 44kB/s  eta 0:00:01    36% |███████████▉                    | 127.4MB 12.9MB/s eta 0:00:17.6MB 52.0MB/s eta 0:00:03    68% |█████████████████████▉          | 235.8MB 11.4MB/s eta 0:00:10    69% |██████████████████████▏         | 238.8MB 10.0MB/s eta 0:00:11
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /home/abevieiramota/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
  Running setup.py bdist_wheel for sacrebleu ... done
  Stored in directory: 

In [11]:
delex = webnlg.load(dataset=['dev_1.2', 'train_1.2'])

with open('paraphrases_delex_src.txt', 'w') as f_src, open('paraphrases_delex_tgt.txt', 'w') as f_tgt:
    
    all_pairs = set()
    
    for entry in delex:
        
        for pair in pairing([l['template'] for l in entry.entry['lexes']]):
            
            all_pairs.add(pair)
            
    for pair in all_pairs:
            
        f_src.write('{}\n'.format(pair[0]))
        f_tgt.write('{}\n'.format(pair[1]))

In [12]:
!head paraphrases_delex_src.txt

AGENT-1 play at BRIDGE-1 in the city of BRIDGE-2 , PATIENT-1 .
AGENT-1 is located in PATIENT-1 in BRIDGE-1 .
AGENT-1 ground is in PATIENT-1 and AGENT-1 compete in PATIENT-2 .
A traditional dish of PATIENT-1 is called AGENT-1 , which has PATIENT-2 as an ingredient and should be served PATIENT-3 .
AGENT-1 , established in PATIENT-3 , is located in BRIDGE-1 , PATIENT-1 and falls under the category of PATIENT-5 . PATIENT-4 is to the north of BRIDGE-1 and to the southeast of BRIDGE-1 lies PATIENT-2 .
PATIENT-4 are an ethnic group in BRIDGE-1 which uses PATIENT-1 . The leader is PATIENT-3 who has the title of PATIENT-2 . BRIDGE-1 is the location of AGENT-1 .
AGENT-1 , completed in PATIENT-2 , has a floor count of PATIENT-1 .
AGENT-1 (PATIENT-3) , published by PATIENT-1 has the LCCN number PATIENT-2 and CODEN code PATIENT-4 .
PATIENT-1 is the area of water in AGENT-1 .
AGENT-1 , PATIENT-2 , has the Architectural style of PATIENT-1 .


In [13]:
!head paraphrases_delex_tgt.txt

AGENT-1 ground is BRIDGE-1 which is located in the city of BRIDGE-2 , PATIENT-1
BRIDGE-1 is in PATIENT-1 . AGENT-1 is located BRIDGE-1 .
AGENT-1 grounds are in PATIENT-1 and obviously AGENT-1 play in PATIENT-2 .
PATIENT-2 is an ingredient of the PATIENT-1 dish AGENT-1 , which should be served PATIENT-3 .
AGENT-1 (location BRIDGE-1 , PATIENT-1) was completed in PATIENT-3 and falls under the category of PATIENT-5 . To the north of BRIDGE-1 lies PATIENT-4 and to the southeast lies PATIENT-2 .
AGENT-1 is located in BRIDGE-1 , where PATIENT-1 is spoken , and one of the ethnic groups is PATIENT-4 . PATIENT-2 is the most important leader , and PATIENT-3 is also a leader .
AGENT-1 was completed in PATIENT-2 and has PATIENT-1 floors .
AGENT-1 was published by PATIENT-1 and is abbreviated PATIENT-3 . The CODEN code is `` PATIENT-4 '' and the LCCN number is PATIENT-2 .
AGENT-1 has an area of water that is PATIENT-1 .
AGENT-1 which is located in PATIENT-2 has the architectural style of PATIENT-1 .

In [16]:
!onmt-build-vocab --size 50000 --save-vocab src-vocab.txt paraphrases_delex_src.txt

Traceback (most recent call last):
  File "/home/abevieiramota/anaconda3/envs/webnlg/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/abevieiramota/anaconda3/envs/webnlg/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/abevieiramota/anaconda3/envs/webnlg/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/abevieiramota/anaconda3/envs/webnlg/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/abevieiramota/anaconda3/envs/webnlg/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.10.0: cannot open shar